In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Conexión a PostgreSQL
user = "userGESTDB"
password = "passGESTDB"
host = "postgres_db"  # nombre del servicio en docker-compose
port = "5432"
db = "GESTDB"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

# Leemos todas las tablas
df_grado = pd.read_sql("SELECT * FROM grado;", engine)
df_universidad = pd.read_sql("SELECT * FROM universidad;", engine)
df_facultad = pd.read_sql("SELECT * FROM facultad;", engine)
df_nota_corte = pd.read_sql("SELECT * FROM nota_corte;", engine)
df_area = pd.read_sql("SELECT * FROM area;", engine)
df_rama = pd.read_sql("SELECT * FROM rama;", engine)
df_grado_ofertado = pd.read_sql("SELECT * FROM grado_ofertado;", engine)

In [ ]:
! pip install rdflib

In [ ]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, XSD, URIRef

# Definir prefijos
sch = Namespace("https://schema.org/")
dat = Namespace("http://example.org/universidadesMadrid#")

g = Graph()
g.bind("sch", sch)
g.bind("dat", dat)
g.bind("xsd", XSD)



In [ ]:
# Ramas -> CategoryCodeSet
for _, row in df_rama.iterrows():
    rama_uri = dat[f"rama_{row['id']}"]
    g.add((rama_uri, RDF.type, sch.CategoryCodeSet))
    g.add((rama_uri, sch.name, Literal(row["nombre"], datatype=XSD.string)))


In [ ]:

# Áreas -> CategoryCode
for _, row in df_area.iterrows():
    area_uri = dat[f"area_{row['id']}"]
    rama_uri = dat[f"rama_{row['id_rama']}"]
    g.add((area_uri, RDF.type, sch.CategoryCode))
    g.add((area_uri, sch.name, Literal(row["nombre"], datatype=XSD.string)))
    g.add((area_uri, sch.inCodeSet, rama_uri))
    g.add((rama_uri, sch.hasCategoryCode, area_uri))



In [ ]:
# Universidades -> CollegeOrUniversity
for _, row in df_universidad.iterrows():
    uni_uri = dat[f"univ_{row['id']}"]
    g.add((uni_uri, RDF.type, sch.CollegeOrUniversity))
    g.add((uni_uri, sch.legalName, Literal(row["nombre"], datatype=XSD.string)))



In [ ]:
# Facultades -> School
for _, row in df_facultad.iterrows():
    fac_uri = dat[f"fac_{row['id']}"]
    uni_uri = dat[f"univ_{row['id_universidad']}"]
    g.add((fac_uri, RDF.type, sch.School))
    g.add((fac_uri, sch.legalName, Literal(row["nombre"], datatype=XSD.string)))
    g.add((fac_uri, sch.telephone, Literal(row["telefono"], datatype=XSD.string)))
    g.add((fac_uri, sch.email, Literal(row["email"], datatype=XSD.string)))
    g.add((fac_uri, sch.url, Literal(row["web"], datatype=XSD.string)))
    g.add((fac_uri, sch.address, Literal(row["direccion"], datatype=XSD.string)))
    g.add((fac_uri, sch.containedInPlace, uni_uri))
    g.add((uni_uri, sch.containsPlace, fac_uri))



In [ ]:
# Grados -> EducationalOccupationalProgram
for _, row in df_grado.iterrows():
    grado_uri = dat[f"grado_{row['id']}"]
    area_uri = dat[f"area_{row['id_area']}"]
    g.add((grado_uri, RDF.type, sch.EducationalOccupationalProgram))
    g.add((grado_uri, sch.name, Literal(row["nombre"], datatype=XSD.string)))
    g.add((grado_uri, sch.description, Literal(row["descripcion"], datatype=XSD.string)))
    g.add((grado_uri, sch.occupationalCategory, area_uri))
    g.add((area_uri, dat.hasOccupationalProgram, grado_uri))



In [ ]:
# Grado ofertado -> CourseInstance
for _, row in df_grado_ofertado.iterrows():
    curso_uri = dat[f"curso_{row['id']}"]
    grado_uri = dat[f"grado_{row['id_grado']}"]
    fac_uri = dat[f"fac_{row['id_facultad']}"]
    g.add((curso_uri, RDF.type, sch.CourseInstance))
    g.add((curso_uri, sch.about, grado_uri))
    g.add((curso_uri, sch.location, fac_uri))


In [ ]:

# Nota de corte -> CutOffScore
for _, row in df_nota_corte.iterrows():
    nota_uri = dat[f"nota_{row['id']}"]
    curso_uri = dat[f"curso_{row['id_grado_ofertado']}"]
    g.add((nota_uri, RDF.type, dat.CutOffScore))
    g.add((nota_uri, dat.educational_program, curso_uri))
    g.add((nota_uri, dat.value, Literal(float(row["nota"]), datatype=XSD.float)))
    g.add((nota_uri, dat.year, Literal(row["año"], datatype=XSD.gYear)))
    g.add((nota_uri, dat.call, Literal(row["convocatoria"], datatype=XSD.string)))
    g.add((nota_uri, dat.group, Literal(row["grupo"], datatype=XSD.string)))



In [ ]:
# Guardar en archivo TTL
g.serialize(destination="universidadesMadrid.ttl", format="turtle")
print("✅ Archivo universidadesMadrid.ttl generado correctamente.")

In [ ]:
df_facultad.head()